In [1]:
from datasets import load_dataset
import pandas as pd
import numpy as np

ds = load_dataset("argilla/FinePersonas-v0.1-clustering-100k")
df = ds['train'].to_pandas()

In [2]:
# Sample 5 random rows to view
df.sample(5)

,id,persona,model_name_embeddings,embedding,nn_indices,nn_scores,projection,cluster_label,summary_label
41298,<urn:uuid:4a6a8152-f708-44c2-8dc5-fbc43f1bfd92>,A science journalist specializing in environme...,Alibaba-NLP/gte-large-en-v1.5,"[-0.026901211589574814, -0.008845492266118526,...","[47336, 38685, 92357, 38973, 53319, 64311, 805...","[0.9561601877212524, 0.9353883862495422, 0.934...","[9.026345252990723, 10.578958511352539]",7,"[""Science"", ""Academia"", ""Space Exploration""]"
72334,<urn:uuid:64724dba-ae88-4708-9f05-b680038b0bb4>,A medical researcher or neuroscientist special...,Alibaba-NLP/gte-large-en-v1.5,"[0.008053791709244251, 0.004684584680944681, -...","[5069, 37049, 78365, 1232, 38107, 73696, 20318...","[0.7844187021255493, 0.7601161599159241, 0.759...","[4.8843159675598145, -1.9842278957366943]",42,"[""Mental Health"", ""Psychology"", ""Healthcare""]"
38154,<urn:uuid:b1edfd62-0f5b-4b1c-a8f3-19ab85dc9b83>,A science educator or a science fair organizer...,Alibaba-NLP/gte-large-en-v1.5,"[-0.017241287976503372, 0.014948854222893715, ...","[27960, 31080, 78701, 30371, 72901, 4510, 8492...","[0.922191321849823, 0.9073317050933838, 0.9038...","[2.4115450382232666, 7.326081275939941]",28,"[""Education"", ""Science"", ""Teaching""]"
51949,<urn:uuid:3bf6bb18-ef32-4c05-8a21-c77e729aa039>,An elementary school teacher focused on litera...,Alibaba-NLP/gte-large-en-v1.5,"[-0.00981831457465887, -0.01127386000007391, -...","[65170, 39893, 22808, 66185, 21736, 33421, 205...","[0.9360370635986328, 0.931273341178894, 0.9223...","[-1.8275816440582275, 3.5921595096588135]",1,"[""Education"", ""Academia"", ""Specialized Experti..."
25648,<urn:uuid:70aed106-87b6-49fd-a74e-13d316545a6b>,A puzzle enthusiast focused on logic-based gri...,Alibaba-NLP/gte-large-en-v1.5,"[-0.0365845263004303, 0.00834557693451643, -0....","[68749, 6409, 6595, 63658, 49497, 29165, 63866...","[0.8998897671699524, 0.8742913007736206, 0.874...","[-1.1516388654708862, 7.8539347648620605]",-1,"""None"""


In [3]:
# # Visualize the 2d embeddings using Plotly
# import plotly.express as px
# import plotly.graph_objects as go

# # Extract the coordinates from the 'projection' column
# df['x'] = df['projection'].apply(lambda x: x[0])
# df['y'] = df['projection'].apply(lambda x: x[1])

# # Create an interactive scatter plot with Plotly
# fig = px.scatter(df, x='x', y='y',
#                  hover_data=['persona', 'summary_label'],
#                  opacity=0.7, title='2D Embeddings Visualization')

# # Customize the layout
# fig.update_traces(marker=dict(size=1, line=dict(width=0.5, color='DarkSlateGrey')))
# fig.update_layout(
#     title_font_size=24,
#     xaxis_title='X Coordinate',
#     yaxis_title='Y Coordinate',
#     width=1200,
#     height=1200,
#     plot_bgcolor='white',
#     paper_bgcolor='white',
#     hovermode='closest',
#     font=dict(family="Arial, sans-serif", size=14, color="black")
# )

# # Improve axis appearance
# fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey', zeroline=False)
# fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey', zeroline=False)

# # Add a subtle border
# fig.update_layout(
#     shapes=[go.layout.Shape(
#         type="rect",
#         xref="paper", yref="paper",
#         x0=0, y0=0, x1=1, y1=1,
#         line=dict(color="Black", width=1)
#     )]
# )

# # Uncomment to show the plot
# fig.show()

In [4]:
# Create a dataframe with the following columns:
# id persona split_id

# Randomly sample 1% of the dataframe for split_id=1, others split_id=0
split_ids = df.sample(frac=0.01, random_state=42).index
df['split_id'] = df.index.isin(split_ids).astype(int)
df_split = df[['id', 'persona', 'split_id']]

# Save the dataframe to a tsv file
data_path = "../data/persona.tsv"
df_split.to_csv(data_path, sep='\t', index=False)


In [5]:
# Verify the number of samples in each split
df_split['split_id'].value_counts()

split_id
0    99000
1     1000
Name: count, dtype: int64

In [6]:
# Get the high-dim and 2d embeddings
high_dim_embeddings = np.array(df['embedding'].to_list())
low_dim_embeddings = np.array(df['projection'].to_list())

# Verify the shapes
print(high_dim_embeddings.shape)
print(low_dim_embeddings.shape)


(100000, 1024)
(100000, 2)


In [7]:
# Save the high-dim and 2d embeddings to a numpy file
np.savez("../data/persona.npz", high_dim_embeddings=high_dim_embeddings, low_dim_embeddings=low_dim_embeddings,)